<a href="https://colab.research.google.com/github/aengdahly/WebscrapingPaginas/blob/main/WebscrapingNature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Webscraping para Revista Nature.

## Paquetes.

In [7]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from google.colab import files

import regex
import warnings
warnings.filterwarnings('ignore')

## Completar la frase.

In [8]:
frase='marine litter'
url_base='https://www.nature.com/search?q={}&order=relevance&article_type=research'.format(frase)

In [9]:
page= requests.get(url_base, verify=False)
html = BeautifulSoup(page.content, 'html')

## Funciones

In [10]:
def EliminarSaltosLinea(oracion):
    string = regex.sub(r'[\n\r\t\xa0\xad]',' ', oracion)
    return(''.join(c for c in string))

def EliminarEspaciosExtras(oracion):
    string = regex.sub(r'\s{2,}',' ', oracion)
    return(''.join(c for c in string))

## Obtener la última página.

In [11]:
paginacion=html.findAll('li',{'class': 'inline-group-item inline-group-middle'})
ult_page=int(paginacion[-2].text.replace('\n\npage',''))

In [ ]:
datos = pd.DataFrame(columns=['Titulo','Pagina','Abstract','Archivo'])

for i in range(1,ult_page+1) : #
    # por cada pàgina
    if i==1:
        url=url_base
    else:
        url=url_base+'&page={}'.format(i)

    page= requests.get(url, verify=False)
    html = BeautifulSoup(page.content, 'html')
    titulos=html.findAll('h2',{'class': 'h3 extra-tight-line-height'})
    links=[z.find('a',{'href': True})['href'] for z in titulos]
  

    for j in links:
      direccion='https://www.nature.com{}'.format(j)
      pagina_articulo=requests.get(direccion, verify=False)
      html2= BeautifulSoup(pagina_articulo.content, 'html')

      titulo=html2.find('h1',{'class':'c-article-title'}).text
      p=html2.find('div',{'class':'c-article-section__content','id':'Abs1-content'})
      if (p!=None):
        abstract=EliminarEspaciosExtras(EliminarSaltosLinea(p.text))
      else:
        abstract='No tiene Abstract como html'  
      archivo=direccion+'.pdf'
      datos = datos.append({'Titulo':titulo,'Pagina':direccion,'Abstract':abstract,'Archivo': archivo}, ignore_index=True)


## Guardar archivo.

In [ ]:
datos.to_csv(frase+'.csv',index=False, sep='|')
files.download(frase+'.csv')